In [ ]:
pip install transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.2 MB/s eta 0:00:00


In [ ]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
import torch

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import numpy as np

# Load a pre-trained Sentence Transformer model
retriever_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def retrieve_documents(query, documents):
    # Encode the query and documents
    query_embedding = retriever_model.encode(query, convert_to_tensor=True)
    doc_embeddings = retriever_model.encode(documents, convert_to_tensor=True)

    query_embedding.to(device)
    doc_embeddings.to(device)

    # Calculate cosine similarity between query and documents
    similarities = cosine_similarity(query_embedding.cpu().detach().numpy().reshape(1, -1), doc_embeddings.cpu().detach().numpy())

    # Get the indices of top 10 most similar documents
    top_n_indices = np.argsort(similarities[0])[-1:][::-1]

    # Get the top n most similar documents
    top_n_documents = [documents[idx] for idx in top_n_indices]

    return top_n_documents


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

wikidata = pd.read_csv("/content/embedding_train_data.csv")
wikidata['info'] = wikidata.apply(lambda row: row['question'] + ' ' + row['content'], axis=1)
wikidata.head()

,id,content,question,info
0,0,1960s \nEarly radio astronomy observations pe...,What was observed in the 1960s by Seth Shostak...,What was observed in the 1960s by Seth Shostak...
1,0,1980s \nA stream of observations in the 1980s...,What were some observations and research condu...,What were some observations and research condu...
2,0,Twentyfirst century \nWhile primordial black ...,What are some recent developments in the theor...,What are some recent developments in the theor...
3,0,In standard cosmology matter is anything whose...,"What is the meaning of the term ""dark matter"" ...","What is the meaning of the term ""dark matter"" ..."
4,0,The arms of spiral galaxies rotate around the ...,What is the conclusion that can be drawn from ...,What is the conclusion that can be drawn from ...


In [ ]:
query = "Can you tell me something about the anatomy of the human body?"
relevant_doc = retrieve_documents(query, wikidata['info'])
print("Query:", query)
print("Most Relevant Document:", relevant_doc)
relevant_doc = '\n'.join(relevant_doc)
#prompt = "Query: "+ query + "\n" + "Relevant documents: " + relevant_doc + '\n' + "Use the information mentioned under Relevant documents to answer the Query mentioned under Query."
prompt ="""
DOCUMENT: {}
QUESTION: {}
INSTRUCTIONS:
Answer the user's QUESTION using the DOCUMENT text above.
Keep your answer grounded in the facts of the DOCUMENT.
If the DOCUMENT doesn’t contain the facts to answer the QUESTION, return {{NONE}}
""".format(relevant_doc, query)
prompt

Query: Can you tell me something about the anatomy of the human body?
Most Relevant Document: ['What are the major anatomical components to understand when learning human anatomy? Humans have the overall body plan of a mammal Humans have a head neck trunk which includes the thorax and abdomen two arms and hands and two legs and feet\nGenerally students of certain biological sciences paramedics prosthetists and orthotists physiotherapists occupational therapists nurses podiatrists and medical students learn gross anatomy and microscopic anatomy from anatomical models skeletons textbooks diagrams photographs lectures and tutorials and in addition medical students generally also learn gross anatomy through practical experience of dissection and inspection of cadavers The study of microscopic anatomy or histology can be aided by practical experience examining histological preparations or slides under a microscopeHuman anatomy physiology and biochemistry are complementary basic medical scie

"\nDOCUMENT: What are the major anatomical components to understand when learning human anatomy? Humans have the overall body plan of a mammal Humans have a head neck trunk which includes the thorax and abdomen two arms and hands and two legs and feet\nGenerally students of certain biological sciences paramedics prosthetists and orthotists physiotherapists occupational therapists nurses podiatrists and medical students learn gross anatomy and microscopic anatomy from anatomical models skeletons textbooks diagrams photographs lectures and tutorials and in addition medical students generally also learn gross anatomy through practical experience of dissection and inspection of cadavers The study of microscopic anatomy or histology can be aided by practical experience examining histological preparations or slides under a microscopeHuman anatomy physiology and biochemistry are complementary basic medical sciences which are generally taught to medical students in their first year at medical 

In [ ]:
import requests
import json

# Your OpenAI API key
api_key = ''

# Endpoint for ChatGPT v1 API
endpoint = 'https://api.openai.com/v1/chat/completions'

def generate_text(prompt, max_tokens=50):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }

    data = {
        'model': 'gpt-3.5-turbo',
        'messages': [
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': prompt}
        ],
        'max_tokens': 2048,  # You can set a maximum token limit
        'temperature': 0.7,
        'top_p': 0.5,
        'frequency_penalty': 0.0,
        'presence_penalty': 0.0,
        'stop': ["\n", ".", "!", "?"],  # Tokens to stop at end of sentences
        'n': 1
    }
    print(data['messages'][1]['content'])
    response = requests.post(endpoint, headers=headers, json=data)
    # print("response : ", response.json())
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        print("Error:", response.text)
        return None

# Example prompt
prompt = prompt

# Generate text
generated_text = generate_text(prompt)

print("Generated Text:")
print(generated_text)


DOCUMENT: What are the major anatomical components to understand when learning human anatomy? Humans have the overall body plan of a mammal Humans have a head neck trunk which includes the thorax and abdomen two arms and hands and two legs and feet
Generally students of certain biological sciences paramedics prosthetists and orthotists physiotherapists occupational therapists nurses podiatrists and medical students learn gross anatomy and microscopic anatomy from anatomical models skeletons textbooks diagrams photographs lectures and tutorials and in addition medical students generally also learn gross anatomy through practical experience of dissection and inspection of cadavers The study of microscopic anatomy or histology can be aided by practical experience examining histological preparations or slides under a microscopeHuman anatomy physiology and biochemistry are complementary basic medical sciences which are generally taught to medical students in their first year at medical sch